# EJERCICIO SIFT

Para la resolución de este ejercicio se propone el código del *script* ```calibrate.py```, el cual puede encontrarse el mismo directorio que este *notebook*. Este *script* se trata de una aplicación reconocimiento de objetos con la *webcam* basada en el número de coincidencias de *keypoints*. Estos *keypoints* no son más que características relevantes de las imágenes que pueden usarse para el reconocimiento de las mismas. Para la extracción de estos puntos, se emplea el algoritmo SIFT, siendo el número de características un valor parametrizable al gusto del usuario. El usuario será capaz de suministrar a la aplicación unos modelos en formato *jpg* ya creados con anterioridad y/o capturar modelos al vuelo con la *webcam*.

Comenzamos con la importación de las librerías requeridas y la definición de ciertas macros y una función auxiliar, *fillborders*, que únicamente se encarga de dibujar los bordes con un color gris en el frame para enmarcar el modelo detectado.

In [2]:
import argparse
import cv2        as cv
import numpy      as np
from glob         import glob
from umucv.stream import Camera
from umucv.util   import putText
from math         import floor


def fillborders(frame,model,x_offset,y_offset,col):
    frame[y_offset:y_offset+model.shape[0],0:x_offset] = col
    frame[y_offset:y_offset+model.shape[0],x_offset+model.shape[1]:2*x_offset+model.shape[1]] = col
    frame[y_offset-x_offset:y_offset,0:2*x_offset+model.shape[1]] = col
    frame[y_offset+model.shape[0]:y_offset+x_offset+model.shape[0],0:2*x_offset+model.shape[1]] = col

NFEATURES = 500  #El numero de keypoints
WH = '640x480'  #Las dimensiones del frame
THRES = 7  #El valor umbral de coincidencias
MODELS_PATH = './SIFT-models/*.jpg'  #El path hacia los modelos jpg
    
W,H = (int(WH.split('x')[0]), int(WH.split('x')[1]))   

if W <= 0 or H <= 0:
    print('Frame dimensions must be positive integers')
    exit()

# Ancho y alto del mini-modelo que solaparemos en el frame original cuando se detecte
modelW,modelH = (floor(W/4.18), floor(H/3.42))
# Offsets para el mini-modelo para solaparlo en el frame
x_offset=10
y_offset=40

A continuación, en caso de que el usuario haya decidido usar sus propios modelos, recorremos el directorio indicado y guardamos en una lista los modelos con las dimensiones indicadas en el bloque de código anterior (vale la pena mencionar que las dimensiones de un modelo y de un frame serán las mismas para garantizar una ejecución más precisa del algoritmo SIFT). Por cada modelo leído, lo almacenamos junto con su nombre y *keypoints* y descriptores que nos proporciona la función ```detectAndCompute``` de SIFT.

In [3]:
sift = cv.xfeatures2d.SIFT_create(nfeatures=NFEATURES)
models = []
files = glob(MODELS_PATH)
for f in files:
    name = f.split('\\')[-1].split('.')[0].replace('_',' ')
    model = cv.resize(cv.imread(f),(W,H))
    kpts, descs = sift.detectAndCompute(model,None)
    models.append((name,model,kpts,descs))

A continuación hacemos uso de la utilidad ```Camera``` de ```umucv``` para la captura de *frames* con hilo. Entramos en un bucle infinito en el que procesamos en cada iteración un *frame*. En cada iteración comprobamos también qué tecla se ha pulsado, cada una mapeándose a las siguientes acciones:

- **ESC** para terminar con el procesamiento.
- **X** para vaciar la colección de modelos.
- **C** para guardar el *frame* actual como modelo. Se actualiza un contador *idx* que se incluye en el nombre para poder distinguir estos modelos capturados en tiempo de ejecución.



```python
cv.namedWindow("camera")
cam = Camera(size = (W,H))
idx = 0
match = cv.BFMatcher()

while True:
    frame = cam.frame.copy()
    
    key = cv.waitKey(1) & 0xFF
    
    if key == 27: 
        break

    if key == ord('x'):
        models.clear()

    if key in [ord('c'), ord('C')]:
        model = frame.copy()
        kpts, descs = sift.detectAndCompute(model,None)
        models.append(("Model_{}".format(idx),model,kpts,descs))
        idx+=1
        print("Added model {}".format(idx))
        
   
    # ...

```

Posteriormente, si disponemos de modelos, realizamos la comparación del *frame* actual con los modelos disponibles. La comparativa debe superar un umbral de coincidencia *THRES* para que el modelo se considere aceptable. Las coincidencias de *keypoints* entre el *frame* actual y el modelo se calculan con el método de ***k-nearest neigbors*** del *matcher* y luego se filtran para dejar solo las que que no tengan ambigüedad. La forma en la que se realiza esto es usando el *ratio test*: se descartan los puntos cuya mejor coincidencia es parecida a la segunda mejor (más concretamente, si no se distingue, como mínimo, en un 85%).

```python
    if models:
            #and key in [ord('e'), ord('E')]

            frame_kpts, frame_descs = sift.detectAndCompute(frame, mask=None)

            modelDetected = np.zeros((modelW,modelW,3),dtype = "uint8")
            best_name = 'None'
            max = 0
            for (name,model,model_kpts,model_descs) in models:
                pts = []

                matches = match.knnMatch(frame_descs,model_descs,k=2)

                #ratio test
                for m in matches:
                    if len(m) >= 2:
                        best,second = m
                        if best.distance < 0.85*second.distance:
                            pts.append(best)

                percent = 100*len(pts)/len(model_kpts)

                #Nos quedamos con el mejor modelo
                if (percent > max):
                    bestModel = (name,model)
                    max = percent

            #Si el porcentaje de coincidencias supera el umbral establecido, se considera el modelo
            if (percent > THRES):
                (name,model) = bestModel
                modelDetected = cv.resize(model.copy(), (modelW,modelW))
                best_name = name

            #Insertamos el modelo detectado en el frame junto con la informacion de coincidencias
            putText(frame, f'{len(frame_kpts)} pts')
            fillborders(frame,modelDetected,x_offset,y_offset,col = (192,192,192))
            frame[y_offset:y_offset+modelDetected.shape[0], x_offset:x_offset+modelDetected.shape[1]] = modelDetected
            putText(frame, f'{percent:.2f}% likelihood', orig = (x_offset,y_offset+modelDetected.shape[0]+30))
            putText(frame, f'{best_name}', orig = (x_offset,y_offset+modelDetected.shape[0]+60))

    #flag = cv.DRAW_MATCHES_FLAGS_DRAW_RICH_KEYPOINTS
    #cv.drawKeypoints(frame, frame_kpts, frame, color=(100,150,255), flags=flag)
    cv.imshow('camera',frame)

cam.stop()
cv.destroyAllWindows()

```

In [4]:
cv.namedWindow("camera")
cam = Camera(size = (W,H))
idx = 0
match = cv.BFMatcher()

while True:
    frame = cam.frame.copy()
    
    key = cv.waitKey(1) & 0xFF
    
    if key == 27: 
        break

    if key == ord('x'):
        models.clear()

    if key in [ord('c'), ord('C')]:
        model = frame.copy()
        kpts, descs = sift.detectAndCompute(model,None)
        models.append(("Model_{}".format(idx),model,kpts,descs))
        idx+=1
        print("Added model {}".format(idx))
        
    if models:
            #and key in [ord('e'), ord('E')]

            frame_kpts, frame_descs = sift.detectAndCompute(frame, mask=None)

            modelDetected = np.zeros((modelW,modelW,3),dtype = "uint8")
            best_name = 'None'
            max = 0
            for (name,model,model_kpts,model_descs) in models:
                pts = []

                matches = match.knnMatch(frame_descs,model_descs,k=2)

                #ratio test
                for m in matches:
                    if len(m) >= 2:
                        best,second = m
                        if best.distance < 0.85*second.distance:
                            pts.append(best)

                percent = 100*len(pts)/len(model_kpts)

                #Nos quedamos con el mejor modelo
                if (percent > max):
                    bestModel = (name,model)
                    max = percent

            #Si el porcentaje de coincidencias supera el umbral establecido, se considera el modelo
            if (percent > THRES):
                (name,model) = bestModel
                modelDetected = cv.resize(model.copy(), (modelW,modelW))
                best_name = name

            #Insertamos el modelo detectado en el frame junto con la informacion de coincidencias
            putText(frame, f'{len(frame_kpts)} pts')
            fillborders(frame,modelDetected,x_offset,y_offset,col = (192,192,192))
            frame[y_offset:y_offset+modelDetected.shape[0], x_offset:x_offset+modelDetected.shape[1]] = modelDetected
            putText(frame, f'{percent:.2f}% likelihood', orig = (x_offset,y_offset+modelDetected.shape[0]+30))
            putText(frame, f'{best_name}', orig = (x_offset,y_offset+modelDetected.shape[0]+60))

    #flag = cv.DRAW_MATCHES_FLAGS_DRAW_RICH_KEYPOINTS
    #cv.drawKeypoints(frame, frame_kpts, frame, color=(100,150,255), flags=flag)
    cv.imshow('camera',frame)

cam.stop()
cv.destroyAllWindows()

640x480 30.0fps
Added model 1
Added model 2
